In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

/Users/john/anaconda3/envs/newenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_name = "tabularisai/multilingual-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [5]:
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 推論関数を定義 (例)
def predict_sentiment(text: str):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_id = torch.argmax(logits, dim=-1).item()
    return predicted_id

# テスト推論
sample_text = "I love this product. It's really amazing!"
sentiment = predict_sentiment(sample_text)
print("Predicted label ID:", sentiment)

Predicted label ID: 4


In [7]:
print(model.config.id2label)

{0: 'Very Negative', 1: 'Negative', 2: 'Neutral', 3: 'Positive', 4: 'Very Positive'}


In [3]:
import os
import glob
import pandas as pd
from transformers import pipeline

def analyze_sentiment_in_csvs(input_pattern: str):
    """
    指定したパターンにマッチするCSVファイルをすべて読み込み、
    'Detail' 列 (index=1) に対して感情分析を行い、
    新たな列 'Sentiment' を追加して、同じファイル名で上書き保存する関数。

    Parameters:
    -----------
    input_pattern : str
        読み込むファイルパスのパターン。
        例: "root/Analyzed/AnalyzedData/analyzed_*_*.csv"
    """

    # 1. Sentiment Analysis パイプラインの準備
    #    多言語の感情分析を想定し、Hugging Faceから公開されているモデルを利用
    #    例: "tabularisai/multilingual-sentiment-analysis"
    sentiment_pipeline = pipeline(
        task="sentiment-analysis",
        model="tabularisai/multilingual-sentiment-analysis"
    )

    # 2. 指定パターンに合うCSVをすべて取得
    csv_files = glob.glob(input_pattern)
    if not csv_files:
        print(f"パターン {input_pattern} に合致するCSVファイルがありません。処理を終了します。")
        return

    for csv_path in csv_files:
        print(f"=== 処理中: {csv_path} ===")

        # 3. CSVを読み込む
        df = pd.read_csv(csv_path)

        # dfのカラム構造:
        #   column[0]: Speaker
        #   column[1]: Detail
        #   column[2]: UniqueWords
        #   column[3]: JLPTLevel
        # → 新たに column[4] (5列目) に感情分析結果を追加

        # 4. Detail列に対して感情分析を適用
        sentiments = []
        for text in df["Detail"]:
            result = sentiment_pipeline(text)
            label = result[0]["label"]  # 例: "positive", "negative", "neutral"など
            sentiments.append(label)

        # 5. 新しい列として追加
        df["Sentiment"] = sentiments

        # 6. 同じファイル名で上書き保存
        #    ※ 元データが必要な場合はバックアップを取るなどご注意ください
        df.to_csv(csv_path, index=False, encoding="utf-8-sig")
        
        print(f"上書き保存完了: {csv_path}\n")


if __name__ == "__main__":
    # 例: "root/Analyzed/AnalyzedData/analyzed_*_*.csv" で
    # analyzed_alpha_{name}.csv, analyzed_beta_{name}.csv を処理する想定。
    pattern = "./Analyzed/AnalyzedData/analyzed_*_*.csv"
    analyze_sentiment_in_csvs(input_pattern=pattern)




Device set to use mps:0


=== 処理中: ./Analyzed/AnalyzedData/analyzed_alpha_Hiraoka.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_alpha_Hiraoka.csv

=== 処理中: ./Analyzed/AnalyzedData/analyzed_beta_Mori.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Mori.csv

=== 処理中: ./Analyzed/AnalyzedData/analyzed_beta_Abe.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Abe.csv

=== 処理中: ./Analyzed/AnalyzedData/analyzed_beta_Suto.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Suto.csv

=== 処理中: ./Analyzed/AnalyzedData/analyzed_beta_Sakai.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Sakai.csv

=== 処理中: ./Analyzed/AnalyzedData/analyzed_beta_Pang.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Pang.csv

=== 処理中: ./Analyzed/AnalyzedData/analyzed_beta_Futamura.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Futamura.csv

=== 処理中: ./Analyzed/AnalyzedData/analyzed_alpha_Mori.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_alpha_Mori.csv

=== 処理中: ./Analyzed/AnalyzedData/analyzed_alph